In [1]:
import json
import os

import scipy.signal as signal

import numpy as np
import pandas as pd

# 导入必要的库，用于读取 JSON 文件、处理信号、处理数据框等

"""
    1. Read tracking data files
"""
# 读取迷路数据文件
path_to_folder = 'Data/'  # 设置数据文件所在的路径
match_id = 18768058  # 比赛的唯一 ID

# 比赛信息文件
with open(f'{path_to_folder}{match_id}.json', 'r', encoding='utf-8') as f:
    match = json.load(f)  # 读取比赛信息 JSON 文件

# 输出比赛信息
print(match)

# 读取跟踪数据文件
data = []
with open(f'{path_to_folder}{match_id}_tracking.json', 'r') as f:
    for line in f:
        obj = json.loads(line)  # 逐行读取跟踪数据
        data.append(obj)  # 将数据添加到列表中

print(len(data))  # 输出数据的长度（帧数）

# 示例读取单个帧数据

# 读取前 5000 帧的跟踪数据
custom_data = []
for d in data[0:5000]: # 测试时只取前 5000 帧数据
    for p in d['players']:
        custom_data.append({
            'frame': int(d['frame_id']),  # 帧 ID
            'period': d.get('period'),  # 比赛时段
            'jersey_number': p.get("jersey_number"),  # 球衣号码
            'speed': p.get('speed', 0),  # 球员速度，默认值为 0
            'player': p.get('person_name'),  # 球员姓名
            'team_name': p.get('team_name'),  # 球队名称
        })

# 转换为数据框
# 将收集到的跟踪数据转换为 Pandas 数据框，并用 0 填充缺失值
df = pd.DataFrame(custom_data).fillna(0)

"""
    Calculate simple match run stats
"""
# 计算比赛中简单的跑动统计数据
column = 'speed'  # 设置要计算的列为速度
match_stats = []
max_speed = 11  # 最大速度设为 11 m/s，约等于 40 km/h
window_length = 7  # 设置平滑窗口长度
use_moving_avg_filter = False  # 设置是否使用移动平均滤波器

# 高速跑动的阈值	hs_run_treshold = 5.4  # 高速跑动的速度阈值为 5.4 m/s，约等于 20 km/h
sprint_window = 1 * 25  # 冲刺窗口时间设置为 1 秒（25 帧）

frames_to_save = []  # 保存感兴趣的帧
for player, df_tracking in df.groupby('player'):  # 按球员分组计算

    # 平滑数据，将速度大于最大速度的设置为 NaN
    df_tracking.at[df_tracking[column] > max_speed] = np.nan

    # 如果使用移动平均滤波器
    if use_moving_avg_filter:
        ma_window = np.ones(window_length) / window_length  # 创建一个移动平均窗口
        df_tracking[column] = np.convolve(df_tracking[column], ma_window, mode='same')
    else:
        # 否则使用 Savitzky-Golay 滤波器来平滑速度数据
        # Savitzky-Golay 滤波器可参考：https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter
        df_tracking[column] = signal.savgol_filter(df_tracking[column], window_length=window_length, polyorder=1)

    # 计算不同速度区间的运动距离
    # 步行（小于 2 m/s）
    walking = df_tracking.loc[df_tracking[column] < 2, column].sum() / 25. / 1000  # 转换为 km/h

    # 慢跑（2 - 4 m/s）
    jogging = df_tracking.loc[ (df_tracking[column] >= 2) & (df_tracking[column] < 4), column].sum() / 25. / 1000

    # 跑步（4 - 7 m/s）
    running = df_tracking.loc[ (df_tracking[column] >= 4) & (df_tracking[column] < 7), column].sum() / 25. / 1000

    # 冲刺（大于等于 7 m/s）
    sprinting = df_tracking.loc[df_tracking[column] >= 7, column].sum() / 25. / 1000

    # 统计数据
    match_stats.append({
        'Player': player,
        'Walking (km/h)': walking,
        'Jogging (km/h)': jogging,
        'Running (km/h)': running,
        'Sprinting (km/h)': sprinting
    })

    # 检查高速跑动的帧
    player_hs_runs = np.diff(1 * (np.convolve(1 * (df_tracking[column] >= hs_run_treshold), np.ones(sprint_window), mode='same') >= sprint_window))
    hs_runs_start = np.where(player_hs_runs == 1)[0] - int(  sprint_window / 2) + 1  # 加上 sprint_window/2 以考虑卷积的中心化
    hs_runs_end = np.where(player_hs_runs == -1)[0] + int(sprint_window / 2) + 1

    frames_to_save.extend(hs_runs_start)

# 转换统计数据为数据框
df_match_stats = pd.DataFrame(match_stats)
print(df_match_stats)

# 任务：计算总距离，按比赛时段和球队分开统计

"""
    2. Exporting Frames of interest
    
"""
# 导出感兴趣的帧
_obj_p = []

for d in data:
    # 检查当前帧是否为感兴趣的帧
    if int(d['frame_id']) in frames_to_save:
        ball = d.get('ball', None)  # 获取球的信息
        if ball is not None:
            _obj_p.append({
                'frame': int(d['frame_id']),  # 帧 ID
                'period': int(d.get('period').split('_')[-1]),  # 比赛时段
                'jersey_number': -99,  # 球衣号码
                'player': "ball",  # 物体为球
                'vx': ball.get('x_velocity', 0),  # x 方向速度
                'vy': ball.get('y_velocity', 0),  # y 方向速度
                'speed': ball.get('speed', 0),  # 球的速度
                'x': ball.get('x', -1),  # x 位置
                'y': ball.get('y', -1),  # y 位置
                'team_name': "ball",  # 球的归属为 "ball"
            })

        for p in d['players']:
            # 获取球员 ID，区分主队和客队
            p_id = f'{"home_team" if p.get("team_name") == match["home_team"]["name"] else "away_team"}_player_{p.get("jersey_number")}'
            _obj_p.append(
                {
                    'frame': int(d['frame_id']),  # 帧 ID
                    'period': int(d.get('period').split('_')[-1]),  # 比赛时段
                    'jersey_number': p.get("jersey_number"),  # 球衣号码
                    'speed': p.get('speed', 0),  # 球员速度
                    'x': p.get('x', -1),  # x 位置
                    'y': p.get('y', -1),  # y 位置
                    'vx': p.get('x_velocity', 0),  # x 方向速度
                    'vy': p.get('y_velocity', 0),  # y 方向速度
                    'player': p.get('person_name'),  # 球员姓名
                    'team_name': p.get('team_name'),  # 球队名称
                }
            )

# 转换为数据框并保存为 parquet 文件
df_tracks = pd.DataFrame(_obj_p).fillna(0)

os.makedirs(f"Data",exist_ok=True)  # 如果目录不存在，则创建目录
df_tracks.reset_index().to_parquet(f"Data/{match_id}_tracks.parquet")  # 导出数据为 parquet 文件格式


FileNotFoundError: [Errno 2] No such file or directory: 'Data/18768058.json'